In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# 1. 数据准备
# 使用IMDB电影评论数据集，这是一个二分类任务（正面评论和负面评论）
max_features = 10000  # 仅使用数据集中前10000个最常见的单词
max_len = 200  # 每个评论的最大长度

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)

# 将每个评论填充/截断为max_len长度
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

In [ ]:

# 2. 实现注意力机制层
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # 初始化权重和偏置
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], input_shape[-1]), initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[-1],), initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # 打分函数，计算每个时间步的分数
        e = K.tanh(K.dot(x, self.W) + self.b)
        # 计算注意力权重，使用softmax函数使权重和为1
        a = K.softmax(e, axis=1)
        # 加权求和，得到最终的输出
        output = x * a
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        # 返回输出的形状
        return input_shape[0], input_shape[-1]

In [ ]:
# 3. 构建和训练模型
model = tf.keras.Sequential()
# 嵌入层，将单词索引转换为密集向量
model.add(tf.keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
# LSTM层，设置return_sequences=True以返回每个时间步的输出
model.add(tf.keras.layers.LSTM(64, return_sequences=True))
# 添加自定义的注意力层
model.add(Attention())
# 输出层，使用sigmoid激活函数进行二分类
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# 编译模型，使用adam优化器和二元交叉熵损失函数
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型，设置训练轮数为5，批次大小为32，并使用20%的数据作为验证集
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# 评估模型在测试集上的性能
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 200, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 200, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention (Attention)           │ (32, 64)               │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,000,901 (15.26 MB)

 Trainable params: 1,333,633 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,667,268 (10.17 MB)